In [ ]:
import requests
import ast
import nltk
from nltk import wordpunct_tokenize
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np

In [ ]:
# construct data
files = ['api_data_usacomp.csv', 'api_data_5+mil.csv', 'api_data_2_5_mil.csv', 'api_data_1_2_mil.csv',
         'api_data_250_500k.csv']#, 'api_data_100_250k.csv']
df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file, names=['City', 'Longitude', 'Latitude', 'Ratings', 
                                         'ObjectNames', 'Description'])
    df = pd.concat([df, data], axis=0)

df = df[df['Description'] != '[]']
df.reset_index(inplace=True)
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = .7,
                            min_df = 1)
td_matrix = vectorizer.fit_transform([x for x in df['Description']])
td_matrix_np = td_matrix.toarray()
td_matrix_np = normalize(td_matrix_np)
docs_compressed, s, words_compressed = svds(td_matrix, k=100)
words_compressed = words_compressed.transpose()
docs_compressed_normed = normalize(docs_compressed)
word_to_index = vectorizer.vocabulary_
index_to_word = {i:t for t,i in word_to_index.items()}
print(docs_compressed.shape,s.shape,words_compressed.shape)

In [ ]:
# driver code
query = input("Type a query: ")
query = vectorizer.transform([query]).toarray()
query_vec = normalize(np.dot(query, words_compressed)).squeeze()
def closest_projects_to_query(query_vec_in, k = 5):
    sims = docs_compressed_normed.dot(query_vec_in)
    asort = np.argsort(-sims)[:k+1]
    return [(i, df['City'][i], sims[i]) for i in asort[1:]]

for i, proj, sim in closest_projects_to_query(query_vec):
    print("{}, {:.4f}".format(proj, sim))

# top_5 = process_query(query, td_mat, city_rev_index, wordpunct_tokenize, ps, stops)
# print("Your ranked destinations:")
# for i in top_5:
#     objects_str = df[df['City'] == i].reset_index()['ObjectNames'][0]
#     descr_str = df[df['City'] == i].reset_index()['Description'][0]
#     ratings_str = df[df['City'] == i].reset_index()['Ratings'][0]
#     objects = ast.literal_eval(objects_str)
#     descriptions = ast.literal_eval(descr_str)
#     ratings = ast.literal_eval(ratings_str)
#     print(i, '- Top Attractions:', objects[np.argmax(ratings)])